In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [ ]:
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 80
pd.options.display.max_rows = 50

In [ ]:
rec1 = pd.read_csv('../data/recipes.csv')

In [ ]:
rec1.rename({'RecipeId': 'id'}, axis=1, inplace=True)

In [ ]:
rec2 = pd.read_csv('../data/recipes_w_search_terms.csv')

In [ ]:
rec_test = pd.merge(rec1, rec2, on='id')

In [ ]:
# Keywords to proper list
rec_test.Keywords = rec_test.Keywords.values.astype(str)
rec_test.Keywords = rec_test.Keywords.str.lower()
rec_test.Keywords = rec_test.Keywords.str.replace('-',' ')
rec_test.Keywords=rec_test.Keywords.str.split('"')
rec_test.Keywords.apply(lambda x: x.remove('c(') if 'c(' in x else x)
rec_test.Keywords.apply(lambda x: x.remove(')') if ')' in x else x)
rec_test.Keywords.apply(lambda x: x.remove('nan') if 'nan' in x else x)
for i in range(2):
    rec_test.Keywords.apply(lambda x: x.remove('') if '' in x else x)
for i in range(35):
    rec_test.Keywords.apply(lambda x: x.remove(', ') if ', ' in x else x)

In [ ]:
# RecipeCategory to proper list
rec_test.RecipeCategory = rec_test.RecipeCategory.values.astype(str)
rec_test.RecipeCategory = rec_test.RecipeCategory.str.lower()
rec_test.RecipeCategory = rec_test.RecipeCategory.str.replace('-',' ')
rec_test.RecipeCategory = rec_test.RecipeCategory.str.split('value that will not be found')
rec_test.RecipeCategory.apply(lambda x: x.remove('nan') if 'nan' in x else x)
None

In [ ]:
# tags to proper list
rec_test.tags = rec_test.tags.astype(str)
rec_test.tags = rec_test.tags.str.lower()
rec_test.tags = rec_test.tags.str.replace('-',' ')
rec_test.tags=rec_test.tags.str.split("'")
rec_test.tags.apply(lambda x: x.remove('[') if '[' in x else x)
rec_test.tags.apply(lambda x: x.remove(']') if ']' in x else x)
rec_test.tags.apply(lambda x: x.remove('nan') if 'nan' in x else x)
rec_test.tags.apply(lambda x: x.remove('') if '' in x else x)
for i in range(80):
    rec_test.tags.apply(lambda x: x.remove(', ') if ', ' in x else x)
None

In [ ]:
# search_terms to proper list
rec_test.search_terms = rec_test.search_terms.astype(str)
rec_test.search_terms = rec_test.search_terms.str.lower()
rec_test.search_terms = rec_test.search_terms.str.replace('-',' ')
rec_test.search_terms = rec_test.search_terms.str.split("'")
rec_test.search_terms.apply(lambda x: x.remove('{') if '{' in x else x)
rec_test.search_terms.apply(lambda x: x.remove('}') if '}' in x else x)
for i in range(20):
    rec_test.search_terms.apply(lambda x: x.remove(', ') if ', ' in x else x)

In [ ]:
# merge lists
rec_test['all_tags'] = rec_test.RecipeCategory+rec_test.Keywords+rec_test.tags+rec_test.search_terms

In [ ]:
# remove duplicates
rec_test.all_tags = rec_test.all_tags.apply(set)
rec_test.all_tags = rec_test.all_tags.apply(list)

In [ ]:
rec_test.all_tags.apply(lambda x: x.remove(' recipes') if ' recipes' in x else x)
rec_test.all_tags.apply(lambda x: x.remove(' recipies') if ' recipies' in x else x)
rec_test.all_tags.apply(lambda x: x.remove('less_thansql:name_topics_of_recipegreater_than') if 'less_thansql:name_topics_of_recipegreater_than' in x else x)
rec_test.all_tags.apply(lambda x: x.remove('throw the ultimate fiesta with this sopaipillas recipe from food.com.') if 'throw the ultimate fiesta with this sopaipillas recipe from food.com.' in x else x)
None

In [ ]:
# Create column Year Published
rec_test['YearPublished']=rec_test.DatePublished.astype(str).str[:4]

In [ ]:
def in_list_replacer(mylist, findvalue, replacewith):
    mylist = np.array(mylist)
    return np.where(mylist == findvalue, replacewith, mylist)

In [ ]:
rec_test.all_tags = rec_test.all_tags.apply(lambda x: in_list_replacer(x,' juice and zest of','juice and zest of'))
rec_test.all_tags = rec_test.all_tags.apply(lambda x: in_list_replacer(x,'1 day or more','> 1 day'))
rec_test.all_tags = rec_test.all_tags.apply(lambda x: in_list_replacer(x,'15 minutes or less','< 15 mins'))
rec_test.all_tags = rec_test.all_tags.apply(lambda x: in_list_replacer(x,'30 minutes or less','< 30 mins'))
rec_test.all_tags = rec_test.all_tags.apply(lambda x: in_list_replacer(x,'4 hours or less','< 4 hours'))
rec_test.all_tags = rec_test.all_tags.apply(lambda x: in_list_replacer(x,'60 minutes or less','< 60 mins'))

In [ ]:
# remove duplicates
rec_test.all_tags = rec_test.all_tags.apply(set)
rec_test.all_tags = rec_test.all_tags.apply(list)

In [ ]:
# list to string
rec_test['all_tags_string'] = [','.join(map(str, l)) for l in rec_test['all_tags']]

In [ ]:
# checkpoint and keep wanted columns
sample = rec_test[['id', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished','YearPublished', 'Description', 'Images',
       'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions',
       'ingredients', 'ingredients_raw_str', 'serving_size',
       'servings', 'steps', 'all_tags_string']]

In [ ]:
# import new columns to be created (make sure it's up to date)
new_cols = pd.read_csv('../data/new_cols.csv')

In [ ]:
# create new columns for filtering
for i in range(len(new_cols)):
    mask = sample['all_tags_string'].str.contains(new_cols.original_tag[i])
    sample.loc[mask, new_cols.column_name[i]]=new_cols.value[i]

In [ ]:
rec = sample.copy()

In [ ]:
rec.Diet_Low_Calories.replace({'Low Calorie':'Low Calories'},inplace=True)

rec.Children_Friendly.replace({'Toddler Friendly':'Toddler Friendly, Kid Friendly'},inplace=True)
rec.Children_Friendly.replace({'Baby Friendly':'Baby Friendly, Toddler Friendly, Kid Friendly'},inplace=True)

rec.Type.replace({'Hot Dog':''},inplace=True)

rec.Diet_Low_Carb.replace({'Very Low Carb':'Very Low Carb, Low Carb'},inplace=True)
rec.Diet_Low_Carb.replace({'Carb Free':'Carb Free, Very Low Carb, Low Carb'},inplace=True)

rec.Diet_Low_Fat.replace({'Low Fat':'Low Fat, Low Saturated Fat'},inplace=True)
rec.Diet_Low_Fat.replace({'Fat Free':'Fat Free, Low Fat, Low Saturated Fat'},inplace=True)

rec.Diet_No_Meat.replace({'Vegetarian':'Vegetarian, No Meat'},inplace=True)
rec.Diet_No_Meat.replace({'Vegan':'Vegan, Vegetarian, No Meat'},inplace=True)

rec.Execution_Time.replace({'Quick':''},inplace=True)
rec.Execution_Time.replace({'< 60 Mins':'< 60 Mins, < 4 Hours'},inplace=True)
rec.Execution_Time.replace({'< 30 Mins':'< 30 Mins, < 60 Mins, < 4 Hours'},inplace=True)
rec.Execution_Time.replace({'< 15 Mins':'< 15 Mins, < 30 Mins, < 60 Mins, < 4 Hours'},inplace=True)

rec.Cuisine.replace({'Us Southern':'North American'},inplace=True)
rec.Cuisine.replace({'Us Southwestern':'North American'},inplace=True)
rec.Cuisine.replace({'Us Midwestern':'North American'},inplace=True)
rec.Cuisine.replace({'Us Northeastern':'North American'},inplace=True)
rec.Cuisine.replace({'American':'North American'},inplace=True)
rec.Cuisine.replace({'Canadian':'North American'},inplace=True)
rec.Cuisine.replace({'Brazilian':'South American'},inplace=True)

In [ ]:
cols=['Type', 'Children_Friendly', 'Cuisine', 'Diet_Cholesterol', 'Diet_Diabetic_Friendly', 'Diet_Egg_Free', 'Diet_Gluten_Free', 'Diet_Healthy', 'Diet_High_Calcium', 'Diet_Lactose_Free', 'Diet_Light', 'Diet_Low_Calories', 'Diet_Low_Carb', 'Diet_Low_Fat', 'Diet_No_Meat', 'Diet_Protein', 'Diet_Sodium', 'Diet_Sugar', 'Easy_Difficulty', 'Execution_Time', 'Menu_Position', 'Season', 'Served_Temperature', 'Spicy', 'Theme', 'Theme_Special', 'Time_Of_Day']

In [ ]:
rec['new_tags'] = rec[cols].apply(lambda x: x.str.cat(sep=', '), axis=1)

In [ ]:
for i in range(30):
    rec.new_tags = rec.new_tags.str.replace(', , ', ', ', regex=True)

In [ ]:
rec.new_tags = rec.new_tags.str.replace('\A, ', '', regex=True)
rec.new_tags = rec.new_tags.str.replace(', \Z', '', regex=True)

In [ ]:
rec=rec[['id', 'Name', 'AuthorId', 'AuthorName', 'CookTime',
       'PrepTime', 'TotalTime', 'DatePublished', 'YearPublished',
       'Description', 'Images', 'RecipeIngredientQuantities',
       'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions',
       'ingredients', 'ingredients_raw_str', 'serving_size', 'servings',
       'steps', 'all_tags_string', 'new_tags']]

In [ ]:
rec['normalized_rating']=rec.AggregatedRating*rec.ReviewCount/rec.ReviewCount.max()

In [ ]:
rec['first_image_url']=rec.Images.str.extract(r'(?<=")(.*?)(?=")')

In [ ]:
# cleaned_2.csv checkpoint
# rec = pd.read_csv('../data/cleaned_2.csv')

In [ ]:
# create unique name column
rec['id_name'] = rec.id.map(str)+'- '+rec.Name

In [ ]:
# columns of recommended daily intake % per serving
rec['calories_daily_perc'] = rec.Calories*100/2000
rec['fat_daily_perc'] = rec.FatContent*100/60
rec['satfat_daily_perc'] = rec.SaturatedFatContent*100/20
rec['chol_daily_perc'] = rec.CholesterolContent*100/300
rec['sodium_daily_perc'] = rec.SodiumContent*100/2300
rec['carbon_daily_perc'] = rec.CarbohydrateContent*100/275
rec['fiber_daily_perc'] = rec.FiberContent*100/35
rec['sugar_daily_perc'] = rec.SugarContent*100/30
rec['protein_daily_perc'] = rec.ProteinContent*100/50

In [ ]:
rec=rec[['id', 'Name', 'id_name',
       'YearPublished', 
       'first_image_url',
       'RecipeInstructions',
       'ingredients_raw_str',
       'new_tags', 'normalized_rating',
       'calories_daily_perc', 'fat_daily_perc', 'satfat_daily_perc',
       'chol_daily_perc', 'sodium_daily_perc', 'carbon_daily_perc',
       'fiber_daily_perc', 'sugar_daily_perc', 'protein_daily_perc']]

In [ ]:
rec.to_csv('cleaned_3.csv', index=False)

In [ ]:
rec.columns